In [2]:
import numpy
import pandas
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import seaborn
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import KMeans
 
seaborn.set_style("white") # change le style par défaut des graphiques seaborn
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'